### Exercise 2 - Create a model to predict customer churn

### 0. Import packages

In [21]:
import os
import gc
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline as sk_pipe
from sklearn.compose import ColumnTransformer

### 1. Data Preparation

#### Read the data

In [22]:
path = '~/Documents/Git/Qustodio-Technical-Test/'
sales = pd.read_csv(os.path.join(path, 'sales.csv'))
user_activity = pd.read_csv(os.path.join(path, 'user_activity.csv'))

In [23]:
sales.head()

,order_id,account_id,start_date,plan,amount,currency,Unnamed: 6
0,C5G1ckzVUC1V,36369294,2019-03-17,MEDIUM,12.95,EUR,NaN
1,LyPKxILXvkiu,36369294,2019-04-17,MEDIUM,12.95,EUR,NaN
2,729R0C9dVx49,36369294,2019-05-17,MEDIUM,12.95,EUR,NaN
3,RrxBXQYG9Qn8,13708705,2020-08-28,SMALL,8.95,EUR,NaN
4,iYemtey2MjLT,940537915,2020-07-17,SMALL,8.95,EUR,NaN


In [24]:
user_activity.head()

,account_id,gender,genre1,genre2,type,games,age,hours
0,101530,male,role-playing,action,mobile,8,21,7.573853
1,731892,female,adventure,action,computer,9,25,4.620231
2,856432,male,action,role-playing,mobile,19,35,13.608988
3,1425820,male,adventure,adventure,mobile,8,20,8.648719
4,1881252,male,action,strategy,computer,6,18,8.929738


#### Data Exploration & Cleaning

In [25]:
sales.dtypes

order_id        object
 account_id      int64
 start_date     object
 plan           object
 amount        float64
 currency       object
Unnamed: 6     float64
dtype: object

Unnamed: 6 column should be dropped and we should change the data type of start_date column. Additionally, some column appear to have a trailing whitespace that needs cleaning.

In [26]:
sales = sales.rename(columns=lambda x: x.strip()) # removes the trailing whitespace
sales['start_date'] = pd.to_datetime(sales.start_date) # transforms start_date into a datetime format
sales.drop(columns= 'Unnamed: 6', inplace= True) # removes the undesired column

In [27]:
sales.describe(include=['O'])

,order_id,plan,currency
count,14788,14788,14788
unique,14788,3,2
top,C5G1ckzVUC1V,SMALL,USD
freq,1,12440,7413


In [28]:
sales.isna().sum()

order_id      0
account_id    0
start_date    0
plan          0
amount        0
currency      0
dtype: int64

No NaN values have been found.

In [29]:
sales[sales.duplicated()]

,order_id,account_id,start_date,plan,amount,currency


No duplicates have been found.

In [30]:
user_activity.dtypes

account_id      int64
gender         object
genre1         object
genre2         object
type           object
games           int64
age             int64
hours         float64
dtype: object

In [31]:
user_activity.describe(include=['O'])

,gender,genre1,genre2,type
count,2000,2000,2000,2000
unique,2,6,6,3
top,male,action,adventure,computer
freq,1400,588,555,1250


In [32]:
user_activity.describe()

,account_id,games,age,hours
count,2.000000e+03,2000.000000,2000.000000,2000.000000
mean,5.073399e+08,8.157500,25.997000,9.134307
std,2.890714e+08,4.880226,6.093238,4.324985
min,1.015300e+05,0.000000,15.000000,0.000000
25%,2.565783e+08,5.000000,22.000000,6.340461
50%,5.040000e+08,8.000000,25.000000,8.635053
75%,7.549967e+08,11.000000,29.000000,11.566813
max,9.996430e+08,33.000000,55.000000,30.000000


In [33]:
user_activity.isna().sum()

account_id    0
gender        0
genre1        0
genre2        0
type          0
games         0
age           0
hours         0
dtype: int64

No NaN values have been found.

In [34]:
user_activity[user_activity.duplicated()]

,account_id,gender,genre1,genre2,type,games,age,hours


No duplicate values have been found.

In [35]:
# set indexes for both dataframes
sales.set_index('order_id', inplace= True)
user_activity.set_index('account_id', inplace= True)

### 2. Create Churn Label

In [36]:
number_of_orders = sales.groupby(by=['account_id']).count()['start_date'].to_frame().reset_index() # checking how many purchases have been made by each user
first_order_date = sales.groupby('account_id')['start_date'].min().reset_index() # extracts the date of the first purchase for each user
last_order_date = sales.groupby('account_id')['start_date'].max().reset_index() # extracts the date of the last purchase for each user

useful_information_sales = number_of_orders.merge(first_order_date, how= 'left', on= 'account_id').merge(last_order_date, how='left', on='account_id')
useful_information_sales.rename(columns= {'start_date_x': 'Orders', 'start_date_y': 'Date_First_Order', 'start_date': 'Date_Last_Order'}, inplace= True) # renames column after the left merge

useful_information_sales = useful_information_sales.merge(sales[['account_id', 'plan', 'currency']], how= 'left', on= 'account_id') # extracts additional useful information from sales dataframe
useful_information_sales.drop_duplicates(inplace=True)
useful_information_sales

,account_id,Orders,Date_First_Order,Date_Last_Order,plan,currency
0,101530,4,2019-09-03,2019-12-03,SMALL,EUR
4,731892,11,2020-02-12,2020-12-12,SMALL,EUR
15,856432,12,2020-01-25,2020-12-25,SMALL,EUR
27,1425820,2,2019-06-25,2019-07-25,SMALL,EUR
29,1881252,1,2019-12-22,2019-12-22,SMALL,USD
...,...,...,...,...,...,...
14739,998101168,3,2020-10-13,2020-12-13,SMALL,EUR
14742,998103378,3,2020-03-14,2020-05-14,SMALL,EUR
14745,998392913,18,2019-07-03,2020-12-03,MEDIUM,USD
14763,999546295,17,2019-03-19,2020-07-19,SMALL,EUR


In [37]:
# build the dataframe to be used to modelling with info from sales.csv and user_actiity.csv
model_df = user_activity.merge(useful_information_sales, how= 'left', on= 'account_id')

assert model_df.shape[0]==user_activity.shape[0], 'Error! There are duplicates in the data'

del useful_information_sales, sales, user_activity # release memory by deleting dataframes that won't be used anymore
gc.collect()

8

In [38]:
model_df.head()

,account_id,gender,genre1,genre2,type,games,age,hours,Orders,Date_First_Order,Date_Last_Order,plan,currency
0,101530,male,role-playing,action,mobile,8,21,7.573853,4,2019-09-03,2019-12-03,SMALL,EUR
1,731892,female,adventure,action,computer,9,25,4.620231,11,2020-02-12,2020-12-12,SMALL,EUR
2,856432,male,action,role-playing,mobile,19,35,13.608988,12,2020-01-25,2020-12-25,SMALL,EUR
3,1425820,male,adventure,adventure,mobile,8,20,8.648719,2,2019-06-25,2019-07-25,SMALL,EUR
4,1881252,male,action,strategy,computer,6,18,8.929738,1,2019-12-22,2019-12-22,SMALL,USD


In [39]:
# Create Test set to do Predictions for Exercise 3
model_df['Date_Last_Order'] = model_df['Date_Last_Order'].dt.to_period('M') # extract only the year and month from the last order date

test_df = model_df[(model_df.Date_Last_Order==model_df.Date_Last_Order.max()) & (model_df.Orders>2) & (model_df.Orders<7)] # saving this dataframe in a variable for predictions
train_df = model_df[~model_df.account_id.isin(list(test_df.account_id.unique()))] # remove prediction users from train set

assert train_df.shape[0] == (model_df.shape[0] - test_df.shape[0]), 'Error! Wrong users have been removed from train_df'

For Exercise 3 we only want to predict on customers that have at least 3 months of order payments and we don't know if they will become churners or not, hence they have to have less than 7 months of order payments.

In [40]:
# Remove users with less than 2 Orders
train_df = train_df[train_df.Orders>2]

# Create Churn Label
train_df['Churn'] = 0
train_df.loc[train_df.Orders<7, 'Churn'] = 1

### 3. Feature Engineering

encoding & normalizing

In [41]:
# Separate labels from features

labels = train_df[['Churn']].copy()
train_df.drop(columns='Churn', inplace= True) # drop Churn feature from train set
len(labels[labels.Churn==1]) / len(labels) * 100 # explore Target variable distribution

21.26899016979446

In [42]:
# Separate Categorical variables from Numerical variables

train_df.set_index('account_id', inplace= True)
train_df_cat = train_df[['gender', 'genre1', 'genre2', 'type', 'plan', 'currency']]
train_df_num = train_df[['games', 'age', 'hours', 'Orders']]

list_cat = train_df_cat.columns.to_list()
list_num = train_df_num.columns.to_list()

In [43]:
# Create the pipelines for both data types

cat_pipepline = sk_pipe([('encoder', OneHotEncoder(categories= 'auto'))])
num_pipeline = sk_pipe([('std_scaler', StandardScaler())])

# Apply the pipelines

pipeline_train_df_cat = cat_pipepline.fit_transform(train_df_cat)
pipeline_train_df_num = num_pipeline.fit_transform(train_df_num)

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, list_num),
    ('cat', cat_pipepline, list_cat),
])

full_pipeline

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('std_scaler',
                                                  StandardScaler())]),
                                 ['games', 'age', 'hours', 'Orders']),
                                ('cat',
                                 Pipeline(steps=[('encoder', OneHotEncoder())]),
                                 ['gender', 'genre1', 'genre2', 'type', 'plan',
                                  'currency'])])

In [44]:
df_train_prepared = full_pipeline.fit_transform(train_df)

In [45]:
df_train_prepared

array([[-0.11047532, -0.97240865, -0.33393489, ...,  1.        ,
         1.        ,  0.        ],
       [ 0.08700373, -0.33374377, -1.00433905, ...,  1.        ,
         1.        ,  0.        ],
       [ 2.06179421,  1.26291841,  1.03590164, ...,  1.        ,
         1.        ,  0.        ],
       ...,
       [ 0.87691992,  2.38058193,  0.03814813, ...,  0.        ,
         0.        ,  1.        ],
       [-1.09787056, -0.17407756, -0.50261505, ...,  1.        ,
         1.        ,  0.        ],
       [ 0.08700373, -0.81274243,  0.63952621, ...,  1.        ,
         0.        ,  1.        ]])

The outcome is a array and we need to transform it back to a dataframe.

In [46]:
cat_cols_encoded = []

# Gets the category for each of the possible labels of categorical variables
for col in list_cat:
    cat_cols_encoded += [f"{col[0]}_{cat}" for cat in list(train_df[col].unique())]

cat_cols_encoded

['g_male',
 'g_female',
 'g_role-playing',
 'g_adventure',
 'g_action',
 'g_strategy',
 'g_sports',
 'g_simulation',
 'g_action',
 'g_role-playing',
 'g_sports',
 'g_strategy',
 'g_adventure',
 'g_simulation',
 't_mobile',
 't_computer',
 't_tablet',
 'p_ SMALL',
 'p_ MEDIUM',
 'p_ LARGE',
 'c_ EUR',
 'c_ USD']

In [54]:
train_df_cat_prepared = pd.DataFrame(pipeline_train_df_cat.toarray(), columns= cat_cols_encoded, 
                                      index= train_df_cat.index) ## transformar para um df as categóricas

train_df_cat_prepared

,g_male,g_female,g_role-playing,g_adventure,g_action,g_strategy,g_sports,g_simulation,g_action,g_role-playing,...,g_adventure,g_simulation,t_mobile,t_computer,t_tablet,p_ SMALL,p_ MEDIUM,p_ LARGE,c_ EUR,c_ USD
account_id,,,,,,,,,,,,,,,,,,,,,
101530,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
731892,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
856432,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2397506,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2436396,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998070356,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
998103378,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
998392913,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [55]:
train_df

,gender,genre1,genre2,type,games,age,hours,Orders,Date_First_Order,Date_Last_Order,plan,currency
account_id,,,,,,,,,,,,
101530,male,role-playing,action,mobile,8,21,7.573853,4,2019-09-03,2019-12,SMALL,EUR
731892,female,adventure,action,computer,9,25,4.620231,11,2020-02-12,2020-12,SMALL,EUR
856432,male,action,role-playing,mobile,19,35,13.608988,12,2020-01-25,2020-12,SMALL,EUR
2397506,male,adventure,role-playing,computer,13,32,7.151723,16,2019-09-06,2020-12,SMALL,USD
2436396,male,adventure,sports,mobile,10,25,14.455147,11,2019-02-18,2019-12,MEDIUM,USD
...,...,...,...,...,...,...,...,...,...,...,...,...
998070356,male,adventure,adventure,mobile,1,25,5.193802,8,2020-05-15,2020-12,SMALL,EUR
998103378,male,adventure,action,mobile,4,21,8.276883,3,2020-03-14,2020-05,SMALL,EUR
998392913,male,adventure,action,computer,13,42,9.213152,18,2019-07-03,2020-12,MEDIUM,USD


### 4. Model Building

### 5. Model Evaluation

### 6. Conclusions

### 7. Predictions